In [1]:
import gurobi as gb
from gurobipy import GRB

import networkx as nx
import numpy as np
import math

In [2]:
from functions_library import *

In [3]:
opt_mod = gb.Model(name="linear programm")
#opt_mod.setParam("IterationLimit",400000)
#opt_mod.setParam("DegenMoves",0);

opt_mod.setParam("MIPFocus",2);

opt_mod.setParam("TimeLimit",600)



Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-02
Set parameter MIPFocus to value 2
Set parameter TimeLimit to value 600


In [4]:
k = 4
G = init_ring(2**k)
D = init_uniformDemand_matrix_symmetric(G)
nodesPairListNoDuplication = complete_node_pair_list_noDuplication(G)
n = 2**k

In [5]:
e = opt_mod.addMVar((n,n), name="e", vtype="B")
SP = opt_mod.addMVar((n,n,n), name="sp", vtype="I", lb=-GRB.INFINITY, ub=(n/2)+1)
helper = opt_mod.addMVar((n,n,n), name="helper", vtype="I", lb=0, ub=1)
opt_mod.update()

In [6]:
# Adjacency Matrix (e) Constraints
opt_mod.addConstrs((e[i,(i+1) % n] == 1 for i in range(n)), name="c-ring")
opt_mod.addConstrs((e[i,i] == 0 for i in range(n)), name="c-noSelfEdge")
opt_mod.addConstrs((e[i,j] == e[j,i] for i in range(n) for j in range(n)), name="c-undirected")

maxNumberE =  k+1 #math.log(n,2)-1 + 2
opt_mod.addConstrs((e[i,:].sum() <= maxNumberE for i in range(n)), name="c-logE")
opt_mod.update() 


opt_mod.addConstrs((SP[0,i,j] == (1-e[i][j])*((n/2)+1)+(e[i][j]) for i in range(n) for j in range(n)), name="c-DP_init")
 
# Ensuring that x shows a correct path
for k in range(1,n):
    for i in range(n):
        for j in range(n):
            if i!=j:
                opt_mod.addGenConstrIndicator(helper[k-1,i,j],True,gb.quicksum([SP[k-1,i,k],SP[k-1,k,j]])>=SP[k-1,i,j])
                opt_mod.addGenConstrIndicator(helper[k-1,i,j],True,SP[k,i,j] == ((gb.quicksum([SP[k-1,i,k],SP[k-1,k,j],SP[k-1,i,j]]))/2)-((gb.quicksum([SP[k-1,i,k],SP[k-1,k,j]])-SP[k-1,i,j])/2))
                opt_mod.addGenConstrIndicator(helper[k-1,i,j],False,SP[k,i,j] == ((gb.quicksum([SP[k-1,i,k],SP[k-1,k,j],SP[k-1,i,j]]))/2)-((SP[k-1,i,j]-gb.quicksum([SP[k-1,i,k],SP[k-1,k,j]]))/2))
                #opt_mod.addConstr(SP[k,i,j] == ((gb.quicksum([SP[k-1,i,k],SP[k-1,k,j],SP[k-1,i,j]]))/2)-(((helper[k-1,i,j]*gb.quicksum([SP[k-1,i,k],SP[k-1,k,j]])+(helper[k-1,i,j]-1)*gb.quicksum([SP[k-1,i,k],SP[k-1,k,j]]))-SP[k-1,i,j])/2))
                #opt_mod.addConstr(SP[k,i,j] <= SP[k-1,i,j])
                #opt_mod.addConstr(helper[k-1,i,j] == gb.quicksum([SP[k-1,i,k],SP[k-1,k,j]]), name="c-DP-helper"), name="c-DP0")
                #pt_mod.addConstr((SP[k-1,i,k] + SP[k-1,k,j]) + n*helper[k-1,i,j] <= SP[k-1,k,j])
                #opt_mod.addConstr(SP[k,i,j] == ((1-helper[k-1,i,j])*(SP[k-1,i,k] + SP[k-1,k,j])) + ((helper[k-1,i,j])*(SP[k-1,i,j])))
                #opt_mod.addConstr(SP[k,i,j] <= SP[k-1,i,k] + SP[k-1,k,j])
                #opt_mod.addConstr((SP[k,i,j] <= SP[k-1,i,j]))
opt_mod.update() 

In [ ]:
#print(len(x[0,1,x,y]))
opt_mod.setObjective(gb.quicksum([SP[n-1,i,j]*D[i][j] for i in G.nodes for j in G.nodes]), GRB.MINIMIZE)
opt_mod.update()


# Run
#opt_mod.display()
opt_mod.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 560 rows, 8448 columns and 1280 nonzeros
Model fingerprint: 0x13ce1171
Model has 10800 general constraints
Variable types: 0 continuous, 8448 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [7e-02, 7e-02]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 9e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve added 16165 rows and 9580 columns
Presolve time: 0.34s
Presolved: 16725 rows, 18028 columns, 52669 nonzeros
Presolved model has 7095 SOS constraint(s)
Variable types: 0 continuous, 18028 integer (6745 binary)
Root relaxation presolve removed 9434 rows and 13834 columns

Root relaxation: numerical trouble, 0 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

In [ ]:
G_sol = nx.Graph()
for i in range(len(G.nodes)): 
    G_sol.add_node(i)

for i in G_sol.nodes:
    for j in range(i+1,len(G_sol.nodes)):
        if e[i,j].X == 1.0:
            G_sol.add_edge(i,j)
pos = nx.circular_layout(G_sol)
nx.draw_networkx(G_sol,pos=pos,with_labels=True)
print(calc_cost(G_sol,D))

In [ ]:
print(G_sol.edges)